## Name entity recognition 

# Author: Hok Seng

<p>
    <b>Notebook Sections</b><br>
    This notebook is divided into the following sections:
    <ul>
        <li>A. Setting Environment</li>
        <li>B. Set the Devic (cpu or mps) </li>
    </ul>
</p>

In [54]:
import os
import subprocess
import requests
import logging
os.getcwd()

'/Users/senghok/Documents/Internship 2A/code'

## Download necessary packages

In [ ]:
# with open("requirement.txt", "r") as file:
#     for f in file.readlines():
#         package = f.strip()
#         if package:
#             subprocess.check_call(["pip", "install", package])
#             %pip install {package}

## Import necessary packages

In [3]:


# save_dir = "data"
# os.makedirs(save_dir,exist_ok = True)

# file_name = os.path.join(save_dir,"NER_dataset.csv")

# response = requests.get(url,stream = True)


# if response.status_code == 200:
#     with open(file_name,"wb") as file:
#         for chunk in response.iter_content(chunk_size=1024):
#             file.write(chunk)
#     print(f"File downloaded successfully and saved to {file_name}")
# else:
#     print(f"Failed to download file. Status code: {response.status_code}")



In [1]:
from gensim.models import Word2Vec

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
import seaborn as sns
import collections
from itertools import chain
import random
import copy
import itertools
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.model_selection import iterative_train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from torch.optim import AdamW
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer
from transformers import BertForTokenClassification
from transformers import get_linear_schedule_with_warmup

import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn import Embedding
from torch.nn import LSTM


# set the option to display all columns
pd.set_option('display.max_columns', None)

/Users/senghok/Documents/Internship 2A/code/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import clean data

In [3]:
train_df = pd.read_csv('./data/train.csv')
valid_df = pd.read_csv('./data/valid.csv')
test_df = pd.read_csv('./data/test.csv')

In [4]:

print(f"Train Dataset size: {len(train_df)}")
print(f"Valid Data Size: {len(valid_df)}")

Train Dataset size: 34279
Valid Data Size: 3808


In [5]:

print("TRAIN Dataset: {}".format(train_df.shape))
print("VALID Dataset: {}".format(valid_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

TRAIN Dataset: (34279, 2)
VALID Dataset: (3808, 2)
TEST Dataset: (9520, 2)


In [6]:
train_df.head()

,sentence,labels
0,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
1,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,The party is divided over Britain 's participa...,"O,O,O,O,O,B-gpe,O,O,O,O,B-geo,O,O,O,O,O,O,O,B-..."
3,Two Germans and four Nigerian oil workers were...,"O,B-gpe,O,O,B-gpe,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,The German firm works as a sub-contractor for ...,"O,B-gpe,O,O,O,O,O,O,B-org,O"


In [7]:
train_df["labels"].iloc[3]

'O,B-gpe,O,O,B-gpe,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,O,B-geo,O,O'

In [8]:
train_df["sentence"].iloc[3]

"Two Germans and four Nigerian oil workers were kidnapped by armed militants during a raid on a boat in Nigeria 's southern oil-rich Delta region ."

In [9]:
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [10]:
train_df.head(3)

,sentence,labels
0,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
1,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,The party is divided over Britain 's participa...,"O,O,O,O,O,B-gpe,O,O,O,O,B-geo,O,O,O,O,O,O,O,B-..."


In [11]:
all_labels = train_df['labels'].apply(lambda x: x.split(','))
all_labels = sorted(set(label for labels in all_labels for label in labels ))
all_labels

['B-geo',
 'B-gpe',
 'B-org',
 'B-per',
 'B-tim',
 'I-geo',
 'I-org',
 'I-per',
 'I-tim',
 'O']

### One hot encoding for all the labels in training , validation and testing dataset

In [14]:

# Step 2: Initialize the MultiLabelBinarizer with all known classes
mlb = MultiLabelBinarizer(classes=sorted(all_labels))
# Step 3: Transform per-sentence label sets
unique_train_labels_ohe = mlb.fit_transform(
    train_df['labels'].apply(lambda x: set(x.split(',')))
)

In [15]:
unique_train_labels_ohe.shape

(34279, 10)

In [14]:


# Validation
unique_valid_labels_ohe = mlb.transform(
    valid_df['labels'].apply(lambda x: set(x.split(',')))
)

# Testing
unique_test_labels_ohe = mlb.transform(
    test_df['labels'].apply(lambda x: set(x.split(',')))
)


In [16]:
# compute number of samples
num_train_samples = unique_train_labels_ohe.shape[0]
num_valid_samples = unique_valid_labels_ohe.shape[0]
num_test_samples = unique_test_labels_ohe.shape[0]

# compute number of samples for each unique label
train_labels_count = unique_train_labels_ohe.sum(axis=0)
valid_labels_count = unique_valid_labels_ohe.sum(axis=0)
test_labels_count = unique_test_labels_ohe.sum(axis=0)

# compute proportion of samples for each unique label
prop_train_labels = train_labels_count/num_train_samples
prop_valid_labels = valid_labels_count/num_valid_samples
prop_test_labels = test_labels_count/num_test_samples


print("Training Data")
print("==============")
print(f"Total num of training samples: {num_train_samples}")
print(f"Total num of each label: {train_labels_count}")
print(f"Proportions of each label: {prop_train_labels}\n")

print("Validation Data")
print("==============")
print(f"Total num of validation samples: {num_valid_samples}")
print(f"Total num of each label: {valid_labels_count}")
print(f"Proportions of each label: {prop_valid_labels}\n")

print("Testing Data")
print("===============")
print(f"Total num of training samples: {num_test_samples}")
print(f"Total num of each label: {test_labels_count}")
print(f"Proportions of each label: {prop_test_labels}")

NameError: name 'unique_valid_labels_ohe' is not defined

### Create the proportion dataFrame for training, validation and testing data

In [12]:
label2id = {k:v+1 for v,k in enumerate(all_labels)}
id2label = {k+1:v for k,v in enumerate(all_labels)}

In [ ]:
# create dict for proportions of each label
prop_data = {
    'labels' : list(label2id.keys()),
    'train' : prop_train_labels,
    'valid' : prop_valid_labels,
    'test' : prop_test_labels
}


prop_df = pd.DataFrame.from_dict(prop_data)


display(prop_df)

In [18]:
# fig, ax = plt.subplots(figsize=(10, 8))
# g = sns.barplot(
#     data=prop_df.melt(id_vars = ["labels"], value_vars=["train", "valid", "test"]),
#     y="value", x="v
# 
# 
# 
# ariable", hue="labels", errorbar=None
# )

# # for legend text
# plt.setp(g.get_legend().get_texts(), fontsize='10')  
 
# # for legend title
# plt.setp(g.get_legend().get_title(), fontsize='10')
# plt.title("Proportions of sentences containing each label - training, validation and testing dataset")
# plt.savefig('./images/label_proportions.png', bbox_inches='tight', dpi=300)
# plt.show()

### Check distributions of 2nd order

In [19]:
# from collections import Counter
# from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

# label_combination_df = pd.DataFrame({
#     'train': Counter(str(combination) for row in get_combination_wise_output_matrix(unique_train_labels_ohe, order=2) for combination in row),
#     'valid': Counter(str(combination) for row in get_combination_wise_output_matrix(unique_valid_labels_ohe, order=2) for combination in row),
#     'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(unique_test_labels_ohe, order=2) for combination in row),
# }).T.fillna(0.0).astype('float')

# # compute proportions
# label_combination_df.loc['train'] = label_combination_df.loc['train']/num_train_samples
# label_combination_df.loc['valid'] = label_combination_df.loc['valid']/num_valid_samples
# label_combination_df.loc['test'] = label_combination_df.loc['test']/num_test_samples

# label_combination_df

In [20]:
# from collections import Counter
# from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

# label_combination_df = pd.DataFrame({
#     'train': Counter(str(combination) for row in get_combination_wise_output_matrix(unique_train_labels_ohe, order=3) for combination in row),
#     'valid': Counter(str(combination) for row in get_combination_wise_output_matrix(unique_valid_labels_ohe, order=3) for combination in row),
#     'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(unique_test_labels_ohe, order=3) for combination in row),
# }).T.fillna(0.0).astype('float')

# # compute proportions
# label_combination_df.loc['train'] = label_combination_df.loc['train']/num_train_samples
# label_combination_df.loc['valid'] = label_combination_df.loc['valid']/num_valid_samples
# label_combination_df.loc['test'] = label_combination_df.loc['test']/num_test_samples

# label_combination_df

### Baseline model
### RNN + Softmax

### Tokenization
There are many tokenization methods  including:
<li>
Bert Tokenizer 
</li>
<li>
Byte-Pair Encoding (BPE) - Robust to unknown words
</li>

### Word tokenization

In [13]:
# Tokenize sentences (whitespace)
def tokenize(sentence):
    return sentence.strip().split()
# Split labels
def split_labels(label_str):
    return label_str.strip().split(',')


In [14]:

train_df['tokens'] = train_df['sentence'].apply(tokenize)
train_df['label_list'] = train_df['labels'].apply(split_labels)

In [15]:
train_df[train_df["tokens"].apply(len) >50]



,sentence,labels,tokens,label_list
600,The visit by Assistant Secretary of State for ...,"O,O,O,O,O,O,B-org,I-org,I-org,I-org,I-org,I-or...","[The, visit, by, Assistant, Secretary, of, Sta...","[O, O, O, O, O, O, B-org, I-org, I-org, I-org,..."
635,Strong performances in construction and manufa...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","[Strong, performances, in, construction, and, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
803,After failing in the Korean War ( 1950 - 53 ) ...,"O,O,O,O,O,O,O,B-tim,I-tim,I-tim,O,O,O,O,B-geo,...","[After, failing, in, the, Korean, War, (, 1950...","[O, O, O, O, O, O, O, B-tim, I-tim, I-tim, O, ..."
1140,"Former U.S. Marine General John J. "" Jack "" Sh...","O,B-org,I-org,I-org,B-per,I-per,I-per,I-per,I-...","[Former, U.S., Marine, General, John, J., "", J...","[O, B-org, I-org, I-org, B-per, I-per, I-per, ..."
4348,But the Honest Man explained that as he was me...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","[But, the, Honest, Man, explained, that, as, h...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...
30696,Mourners and officials of the Socialist Party ...,"O,O,O,O,O,B-org,I-org,O,O,B-per,O,O,O,O,O,O,O,...","[Mourners, and, officials, of, the, Socialist,...","[O, O, O, O, O, B-org, I-org, O, O, B-per, O, ..."
30796,In an interview broadcast Sunday on U.S. telev...,"O,O,O,O,B-tim,O,B-geo,O,O,O,B-org,O,O,O,O,O,B-...","[In, an, interview, broadcast, Sunday, on, U.S...","[O, O, O, O, B-tim, O, B-geo, O, O, O, B-org, ..."
32287,Following its heyday as a global maritime powe...,"O,O,O,O,O,O,O,O,O,O,B-tim,I-tim,I-tim,O,O,B-pe...","[Following, its, heyday, as, a, global, mariti...","[O, O, O, O, O, O, O, O, O, O, B-tim, I-tim, I..."
32874,"Renewed activity in the mining sector , the so...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,O,O,O,...","[Renewed, activity, in, the, mining, sector, ,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


### ID tokenization

### We can use Text vectorization

# Text vectorization consiste of parts:
<li>
    Standardization
</li>
<li>
    Tokenization
</li>
<li>
    Vocabulary building
</li>

### Vocabulary

### Necessary dictionaries for encoding and decoding phase

In [16]:

label2id = {k:v+1 for v,k in enumerate(all_labels)}
id2label = {k+1:v for k,v in enumerate(all_labels)}

In [17]:
label2id["<PAD>"] = 0

In [18]:
label2id

{'B-geo': 1,
 'B-gpe': 2,
 'B-org': 3,
 'B-per': 4,
 'B-tim': 5,
 'I-geo': 6,
 'I-org': 7,
 'I-per': 8,
 'I-tim': 9,
 'O': 10,
 '<PAD>': 0}

In [19]:
word_counts = Counter(word for tokens in train_df['tokens'] for word in tokens)
word2id = {word: i + 2 for i, word in enumerate(word_counts)} # turn words into unique ids (+2) because give 0,1 to UNK and PAD

In [20]:
id2word = {i+2:word for i,word in enumerate(word_counts)}


### Add '<PAD>' to use when padding and '<UNK>' in the case where there are unknown entities

In [21]:
word2id['<PAD>'] = 0
word2id['<UNK>'] = 1

In [30]:
fallback = '<UNK>'
fallback_id = word2id.get('B-gpe', list(label2id.values())[0])  ## get idea of a word

fallback_id

1

In [31]:
pad_token = '<PAD>'
pad_id = label2id.get(pad_token, list(label2id.values())[0])  
pad_id

0

In [21]:
MAX_LEN = 40

def encode(tokens, vocab, pad_len=40, fallback='<UNK>', pad_token='<PAD>'):
    """
    Encodes a list of tokens using a vocabulary.

    - tokens: List[str] → tokens or labels (e.g., words or NER tags)
    - vocab: dict → maps token to index
    - pad_len: int → fixed output length (pad or truncate)
    - fallback: str → token to use if one isn't found (only relevant for input)
    - pad_token: str → token to use for padding

    Returns: List[int] of length `pad_len`
    """
    fallback_id = vocab.get(fallback, 0)     # Use <UNK> if token not found
    pad_id = vocab.get(pad_token, 0)         # Use <PAD> for padding

    # Map each token to its ID
    ids = [vocab.get(tok, fallback_id) for tok in tokens]

    # Truncate or pad to fixed length
    if len(ids) > pad_len:
        return ids[:pad_len]
    else:
        return ids + [pad_id] * (pad_len - len(ids))


In [22]:
X = np.array([encode(tokens, word2id, fallback='<UNK>', pad_token='<PAD>') for tokens in train_df['tokens']])
y = np.array([encode(labels, label2id, fallback='O', pad_token= '<PAD>') for labels in train_df['label_list']])
vocab_size = len(word2id)
label_size = len(label2id)


In [23]:
X[0]

array([ 2,  3,  4,  5,  6,  7,  8,  9,  7, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 18, 23, 18, 24,  7, 25, 26, 18,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0])

In [24]:
## Example:
sentence = train_df['tokens'][2]   
label    = train_df['label_list'][2]
encoded_sentence = encode(sentence,word2id,fallback= '<UNK>', pad_token = '<PAD>')
encoded_label  = encode(label, label2id, fallback = 'O', pad_token = '<PAD>')
print(f"Sentence : {sentence} with length {len(sentence)}\n",
       f"encoded sentence : {encoded_sentence} with length {len(encoded_sentence)}\n")

print(f"Label : {label} with length {len(label)}\n",
       f"encoded label : {encoded_label} with length {len(encoded_label)}")

Sentence : ['The', 'party', 'is', 'divided', 'over', 'Britain', "'s", 'participation', 'in', 'the', 'Iraq', 'conflict', 'and', 'the', 'continued', 'deployment', 'of', '8,500', 'British', 'troops', 'in', 'that', 'country', '.'] with length 24
 encoded sentence : [39, 40, 41, 42, 43, 44, 45, 46, 6, 7, 47, 8, 23, 7, 48, 49, 3, 50, 51, 52, 6, 53, 54, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] with length 40

Label : ['O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O'] with length 24
 encoded label : [10, 10, 10, 10, 10, 2, 10, 10, 10, 10, 1, 10, 10, 10, 10, 10, 10, 10, 2, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] with length 40


In [25]:

id2word = {v:k for k,v in word2id.items()}
# Decode the sentence
decoded_sentence = [id2word.get(word_id, '<UNK>') for word_id in encoded_sentence]
decoded_sentence

['The',
 'party',
 'is',
 'divided',
 'over',
 'Britain',
 "'s",
 'participation',
 'in',
 'the',
 'Iraq',
 'conflict',
 'and',
 'the',
 'continued',
 'deployment',
 'of',
 '8,500',
 'British',
 'troops',
 'in',
 'that',
 'country',
 '.',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>']

In [26]:
## Convert to sentence with masked entity
sentence_masked = []
for index, lab in enumerate(label):
    if lab == 'O':
        sentence_masked.append(sentence[index])
    else:
        sentence_masked.append(label[index])
sentence_masked       

['The',
 'party',
 'is',
 'divided',
 'over',
 'B-gpe',
 "'s",
 'participation',
 'in',
 'the',
 'B-geo',
 'conflict',
 'and',
 'the',
 'continued',
 'deployment',
 'of',
 '8,500',
 'B-gpe',
 'troops',
 'in',
 'that',
 'country',
 '.']

In [27]:
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (34279, 40), y shape: (34279, 40)


In [29]:
vocab_size,label_size

(30523, 11)

In [30]:
valid_df

,sentence,labels
0,They marched from the Houses of Parliament to ...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O"
1,The International Atomic Energy Agency is to h...,"O,B-org,I-org,I-org,I-org,O,O,O,O,O,O,O,O,B-ge..."
2,"The European Union , with U.S. backing , has t...","O,B-org,I-org,O,O,B-gpe,O,O,O,O,O,O,B-gpe,O,O,..."
3,Militant groups frequently attack oil operatio...,"O,O,O,O,O,O,O,O,B-geo,I-geo,O,O,O,O,O,O,O,O,O,..."
4,The attacks occurred after the government said...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-gpe,..."
...,...,...
3803,Aircraft also struck a building allegedly used...,"B-org,O,O,O,O,O,O,O,O,O,O,O,O"
3804,One Hamas militant was reported killed and two...,"O,B-org,O,O,O,O,O,O,O,O,O,O"
3805,"Media reports say Ford will cut 25,000 or more...","O,O,O,B-org,O,O,O,O,O,O,O,O,O,B-tim,I-tim,I-ti..."
3806,The powerful Muslim Brotherhood Movement based...,"O,O,B-org,I-org,I-org,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [28]:
# Tokenize sentences and split labels
for df in [valid_df, test_df]:
    df['tokens'] = df['sentence'].apply(lambda x: x.strip().split())
    df['label_list'] = df['labels'].apply(lambda x: x.strip().split(','))
X_valid = np.array([
    encode(tokens, word2id, fallback='<UNK>', pad_token='<PAD>')
    for tokens in valid_df['tokens']
])

y_valid = np.array([
    encode(labels, label2id, fallback='O', pad_token='O')
    for labels in valid_df['label_list']
])
X_test = np.array([
    encode(tokens, word2id, fallback='<UNK>', pad_token='<PAD>')
    for tokens in test_df['tokens']
])
y_test = np.array([
    encode(labels, label2id, fallback='O', pad_token='O')
    for labels in test_df['label_list']
])


In [29]:
print(f"Shape of Training Data (X, y): {X.shape}, {y.shape}")
print(f"Shape of Validation Data (X_valid, y_valid): {X_valid.shape}, {y_valid.shape}")
print(f"Shape of Test Data (X_test, y_test): {X_test.shape}, {y_test.shape}")

Shape of Training Data (X, y): (34279, 40), (34279, 40)
Shape of Validation Data (X_valid, y_valid): (3808, 40), (3808, 40)
Shape of Test Data (X_test, y_test): (9520, 40), (9520, 40)


In [30]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)
X_valid = torch.from_numpy(X_valid)
y_valid = torch.from_numpy(y_valid)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [31]:
X.shape

torch.Size([34279, 40])

### Embedding Layer

### Baseline model

In [32]:
class NERDataset(Dataset):
    def __init__(self,X_tensor,y_tensor,lengths):
        self.X = X_tensor
        self.y = y_tensor
        self.lengths = lengths
        
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self,idx):
        return torch.tensor(self.X[idx],dtype= torch.long), \
                torch.tensor(self.y[idx], dtype = torch.long),\
                torch.tensor(self.lengths[idx], dtype = torch.long)
#pack individual examples into a single batch
def collate_fn(batch):  
    tokens, tags,lengths= zip(*batch)
    tokens = torch.stack(tokens)
    tags = torch.stack(tags)
    lengths = torch.stack(lengths)
    return tokens, tags, lengths
    

In [62]:
example = torch.randint(10, (2, 2))
y = torch.randint(5,(2,))
length = [len(a) for a in example]

### For debugging

In [33]:
from IPython.core.debugger import set_trace
import pdb
import tqdm

### Parameters

In [34]:
PAD_IDX = label2id['<PAD>']   # 0
VOCAB_SIZE = vocab_size  # 30523
TAGSET_SIZE = len(label2id)
BATCH_SIZE = 32
NUM_EPOCHS = 10
learning_rate = 0.001

#orignal length sequence:


In [35]:
class BiLSTMTagger(torch.nn.Module):
    def __init__(self,
                 tagset_size: int,
                 embedding_dim: int = 200,
                 hidden_dim: int = 128,
                 num_layers: int = 1,
                 dropout: float = 0.3,
                 pad_idx: int = 0):  #pad_idx = vocab["<pad>"]
        super().__init__()  # python 3+ , if python 2+ then , super(BiLSTMTagger,self)
        
        self.embedding = nn.Embedding(VOCAB_SIZE, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,  ## hidden backward + hidden forward (h_tf+ h_tb)  # single then remove 2 , 
            
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        
        self.dropout = nn.Dropout(dropout)
        # each time step we pass the output of LSTM to a fully connected layer 
        self.fc = nn.Linear(hidden_dim *2, tagset_size) # tagset_size = 10
        self.softmax = nn.Softmax(dim=-1)  

    def forward(self, x: torch.LongTensor,lengths: torch.LongTensor):
        # exercice: put the shape of each layer
        emb = self.embedding(x)  # (B,S,E_dim)
        # Pack to let LSTM skip the padded tokens
        # there are some sequences whose lengths are greater than 40(padding sequence)
        lengths = torch.clamp(lengths, max = x.size(1))
        
        packed = pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False) #(T_total, Emb) (sum of actual sequence length)
        # LSTM
        packed_out,_ = self.lstm(packed)  # (T_total,Hidden_dimension*2(2H)) Hidden state for each time step  # Pytorch initialize the initial state to zero
        lstm_out,_ = pad_packed_sequence(packed_out, batch_first=True, total_length=x.size(1))  # (B,S,2H) Ignore o, c, etc
        
        out = self.dropout(lstm_out)  #(B,S,2H)
        logits = self.fc(out) #(B,S,C)
        return logits

### Batching training data

In [36]:

lengths_train = np.array([len(seq) for seq in train_df["tokens"]])
lengths_valid = np.array([len(seq) for seq in valid_df["tokens"]])
lengths_test = np.array([len(seq) for seq in test_df["tokens"]])
dataset_train = NERDataset(X,y,lengths_train)
loader_train = DataLoader(dataset_train,
                    batch_size = 32,
                    shuffle = True,
                    collate_fn = collate_fn,
                    num_workers =0, # of CPU
                    pin_memory = False)  # true if we use GPU



In [37]:
# validation dataset 

dataset_valid = NERDataset(X_valid,y_valid, lengths_valid)
loader_valid = DataLoader(dataset_valid,
                    batch_size = 32,
                    shuffle = True,
                    collate_fn = collate_fn,
                    num_workers =0, # of CPU
                    pin_memory = False)

In [38]:
# test dataset 
dataset_test = NERDataset(X_test,y_test, lengths_test)
loader_test = DataLoader(dataset_test,
                    batch_size = 32,
                    shuffle = True,
                    collate_fn = collate_fn,
                    num_workers =0, # of CPU
                    pin_memory = False)

### Optimizer and Device

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiLSTMTagger(VOCAB_SIZE,TAGSET_SIZE).to(device)
criterion = nn.CrossEntropyLoss(ignore_index= PAD_IDX)
#nn.CrossEntropyLoss—it expects raw logits and internally does a log_softmax followed by an NLL loss.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [66]:
PAD_IDX = 0

# Predicted labels (flattened from a batch of sequences)
preds = torch.tensor([1, 2, 0, 3, 2, 0])

# True labels
labels = torch.tensor([1, 2, 0, 4, 2, 0])
mask = labels != PAD_IDX
# Output: tensor([True, True, False, True, True, False])
preds == labels
# Output: tensor([True, True, True, False, True, True])
correct = (preds == labels) & mask
# Output: tensor([ True,  True, False, False,  True, False])

mask

tensor([ True,  True, False,  True,  True, False])

In [67]:
X.shape

torch.Size([34279, 40])

### Evaluation metric

In [40]:
# ## for tmr: 
# ## write the accuracy function to put in the training loop
# average accuracy 
def accuracy(pred,labels,pad_idx):
    mask = labels != pad_idx
    correct = (pred == labels) & mask
    return correct.sum().item()/mask.sum().item()


In [41]:
X_valid[0]

tensor([ 1754,  3225,   138,     7, 24721,     3,  3410,   110,    67,  1491,
            6,  7972,  3287,    26,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

### Test for overfitting

## EarlyStopping Class

We can implement an `EarlyStopping` class to control the training phase if the validation loss stops improving.

### Parameters

- **`patience`**  
  Number of epochs to wait before stopping if no improvement.
- **`delta`**  
  Minimum change in the monitored quantity to qualify as an improvement.
- **`best_score`**, **`best_model_state`**  
  Track the best validation score and model state.

### Methods

- **`__call__(val_loss, model)`**  
  Updates the early stopping logic.


### Regularization

In [42]:
class EarlyStopping:
    def __init__(self,epoch_limit = 5, improv_threshold = 0):
        self.epoch_limit = epoch_limit
        self.improv_threshold = improv_threshold  #minimum change
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_state = None
    # create our own callable to facilitate configuration
    def __call__(self, val_loss, model):
        score = val_loss
        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict()  #keep track of the model state (parameters(weights and biases))
        elif score > self.best_score + self.improv_threshold:
            self.counter += 1
            if self.counter >= self.epoch_limit:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict()
            self.counter = 0  # reset counter to zero
            
    def load_best_model(self, model):
        model.load_state_dict(self.best_model_state)
   

In [146]:
# ## example:

# early_stop = EarlyStopping(epoch_limit = 4, improv_threshold = 1)

# loss = 4
# for i in range(4):
#     loss += 0.5
#     early_stop(loss)
#     if early_stop.early_stop:
#         break

In [43]:
early_stopping = EarlyStopping(epoch_limit=5, improv_threshold=0.01)

In [ ]:
from tqdm import tqdm
torch.manual_seed(42)

for epoch in range(NUM_EPOCHS):
    model.train()

    train_loss = 0.0
    for batch_tokens, batch_tags, lengths in loader_train :  # tuple from the collate_fn function 
        batch_tokens = batch_tokens.to(device) #unecessary here since it is automatic
        lengths = lengths.to(device)
        batch_tags = batch_tags.to(device)
        
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_tokens,lengths)  # (B, S, C)  C = 10
        # Flatten logits and targets for loss computation
        logits = logits.reshape(-1, logits.size(-1))  # (B*S, C)
        batch_tags = batch_tags.view(-1) #true value             # (B*S)
        # Compute loss
        loss = criterion(logits, batch_tags)
        # Regularization to generalize the model or to escape overfitting
        l2_lambda = 0.001
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        loss = loss +l2_lambda*l2_norm
            
        # Backpropagation with anomaly detection
        with torch.autograd.set_detect_anomaly(True):
            loss.backward()
                
        optimizer.step()
        train_loss += loss.item()
        
        
         
    #test on validation dataset
    val_loss = 0.0
    valid_accuracy = 0
    num_batches = 0
    model.eval()
    with torch.inference_mode():
        for tokens, tags, lengths_valid in loader_valid:
            tokens = tokens.to(device)
            tags = tags.to(device)
            lengths_valid = lengths_valid.to(device)
            
            output = model(tokens,lengths_valid)  #shape (B,S,C)
            output_flat = output.view(-1,output.size(-1))  #(B*S,C)
            tags_flat = tags.view(-1)  # shape (B*S,)
            loss = criterion(output_flat,tags_flat)
            val_loss += loss.item()
            prediction_valid = torch.argmax(output,dim =-1)
            valid_accuracy += accuracy(prediction_valid,tags,pad_idx = PAD_IDX)
            num_batches += 1
            
    valid_accuracy = valid_accuracy/num_batches 
      
    print(f"Epoch {epoch+1} — Train_Loss: {train_loss:.4f}, Valid_Loss: {val_loss:.4f}, Accuracy_valid: {valid_accuracy:.4f}")  
    with open("training_log.txt", "a") as f:
        f.write(f"Epoch {epoch+1} — Train_Loss: {train_loss:.4f}, Valid_Loss: {val_loss:.4f}, Accuracy_valid: {valid_accuracy:.4f}\n")
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

early_stopping.load_best_model(model)   
   


### Save learnable parameters 

In [107]:
# torch.save(model.state_dict()["embedding.weight"], "embedding_weight.pth")
# embedding_weight = torch.load("embedding_weight.pth")
# embedding_weight

In [99]:
# #code that work:
# from tqdm import tqdm
# import torch

# torch.manual_seed(42)

# for epoch in range(1, NUM_EPOCHS + 1):
#     # ——————— Training ———————
#     model.train()
#     train_loss = 0.0
#     for batch_tokens, batch_tags, lengths in loader_train:
#         batch_tokens = batch_tokens.to(device)
#         batch_tags   = batch_tags.to(device)
#         lengths      = lengths.to(device)

#         optimizer.zero_grad()
#         # raw logits: (B, S, C)
#         logits = model(batch_tokens, lengths)

#         # flatten for loss: (B*S, C) and (B*S,)
#         logits_flat = logits.view(-1, logits.size(-1))
#         tags_flat   = batch_tags.view(-1)

#         loss = criterion(logits_flat, tags_flat)
#         with torch.autograd.set_detect_anomaly(True):
#             loss.backward()
#         optimizer.step()

#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(loader_train)

#     # ————— Validation —————
#     model.eval()
#     valid_loss = 0.0
#     valid_acc  = 0.0
#     n_batches  = 0

#     with torch.inference_mode():
#         for tokens, tags, lengths_valid in loader_valid:
#             tokens = tokens.to(device)
#             tags   = tags.to(device)
#             lengths_valid = lengths_valid.to(device)

#             # raw logits: (B, S, C)
#             output = model(tokens, lengths_valid)

#             # loss on flattened
#             out_flat = output.view(-1, output.size(-1))
#             tags_flat = tags.view(-1)
#             loss = criterion(out_flat, tags_flat)
#             valid_loss += loss.item()

#             # predictions for accuracy
#             preds = torch.argmax(output, dim=-1)       # (B, S)
#             valid_acc += accuracy(preds.view(-1), tags_flat, pad_idx=PAD_IDX)
#             n_batches += 1

#     avg_valid_loss = valid_loss / n_batches
#     avg_valid_acc  = valid_acc  / n_batches

#     print(
#         f"Epoch {epoch}/{NUM_EPOCHS} — "
#         f"Train Loss: {avg_train_loss:.4f}, "
#         f"Valid Loss: {avg_valid_loss:.4f}, "
#         f"Valid Acc: {avg_valid_acc:.4f}"
#     )

#     # early stopping
#     early_stopping(avg_valid_loss, model)
#     if early_stopping.early_stop:
#         print("Early stopping")
#         break

# # load best model after training
# early_stopping.load_best_model(model)


In [44]:
import torch
import torch.nn.functional as F

In [82]:
logits = torch.tensor([2.0, 1.0, 0.1])  # raw result from the pipeline
probs = F.softmax(logits)

/var/folders/_9/m2ryk39s2slc67mc6y5xdxcm0000gn/T/ipykernel_42154/3044564666.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)


In [119]:
from seqeval.metrics import classification_report as seqeval_report

In [120]:
from sklearn.metrics import classification_report

# Predict
y_pred_test = model.predict(X_test)
y_pred_ids_test = np.argmax(y_pred_test, axis=-1)   #compare the distribution of the output of softmax for a token-word (10 scores comparison)
y_true_ids_test = y_test

# Flatten and filter padding
true_flat, pred_flat = [], []
for i in range(len(y_true_ids_test)):
    for true, pred in zip(y_true_ids_test[i], y_pred_ids_test[i]):
            true_flat.append(id2label[true])
            pred_flat.append(id2label[pred])

# Report
print(classification_report(true_flat, pred_flat, digits=4))


298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
              precision    recall  f1-score   support

       B-geo     0.8036    0.8564    0.8291      7491
       B-gpe     0.9411    0.9037    0.9220      3166
       B-org     0.7218    0.5719    0.6382      4006
       B-per     0.8241    0.7663    0.7941      3350
       B-tim     0.9009    0.8156    0.8562      3992
       I-geo     0.7437    0.7974    0.7696      1441
       I-org     0.8073    0.7057    0.7531      3259
       I-per     0.8663    0.8020    0.8329      3450
       I-tim     0.7883    0.6516    0.7135      1286
           O     0.9909    0.9962    0.9936    349359

    accuracy                         0.9781    380800
   macro avg     0.8388    0.7867    0.8102    380800
weighted avg     0.9773    0.9781    0.9775    380800



In [ ]:
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

y_test_flat = y_test.flatten()
y_pred_test_probs = y_pred_test.reshape(-1, y_pred_test.shape[-1])

fpr = {}
tpr = {}
roc_auc = {}
for i in range(label_size):
    fpr[i], tpr[i], _ = roc_curve((y_test_flat == i).astype(int), y_pred_test_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve
plt.figure(figsize=(10, 8))
for i in range(label_size):
    plt.plot(fpr[i], tpr[i], label=f'Class {id2label[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Test Dataset')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:

# Get all unique labels from your label map
all_labels = sorted(list(set(true_flat + pred_flat)))

cm = confusion_matrix(true_flat, pred_flat, labels=all_labels)


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=all_labels, yticklabels=all_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [125]:
def show_predictions(df, X, y_true_ids, y_pred_ids, index=0):
    print(f"\nSentence: {' '.join(df.loc[index, 'tokens'])}")
    print(f"{'Token':15} {'True':10} {'Predicted'}")
    print("-" * 40)
    for token, true, pred in zip(df.loc[index, 'tokens'], 
                                 [id2label[i] for i in y_true_ids[index]],
                                 [id2label[i] for i in y_pred_ids[index]]):
        print(f"{token:15} {true:10} {pred}")


In [ ]:
show_predictions(test_df, X_test, y_true_ids_test, y_pred_ids_test, index=22)

### Use pretrained embedding


We encounter OOV(out-of-verb) problem  

### Word2Vec

We need to use : GoogleNews-vectors-negative300.bin.gz
Corpus that has 100 billion words. It uses the skip gram Word2Vec model. It uses negative sampling. Each word is represented by a 300 dimension vector.

In [48]:
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings # to prevent displaying any warnings
from gensim.models import KeyedVectors
import gensim.downloader as api

In [49]:

word2vec_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Example: Get the embedding vector for a word
word = "example"  # Replace with the word you want
if word in word2vec_model:
    embedding_vector = word2vec_model[word]
    print(f"Embedding vector for '{word}':\n{embedding_vector}\n")
    print(f"Dimension of the embedding vector:{len(embedding_vector)}")
    


In [53]:
model = gensim.models.Word2Vec(train_df["tokens"], min_count = 1, vector_size = 100, window = 5, sg = 1)


(100,)

In [52]:
# skip gram
def Word2Vec(token):  #df[tokens].iloc[]
    model = gensim.models.Word2Vec(train_df["tokens"], min_count = 1, vector_size = 100, window = 5, sg = 1)
    return model.
    
    
    

0        [Families, of, soldiers, killed, in, the, conf...
1        [Police, put, the, number, of, marchers, at, 1...
2        [The, party, is, divided, over, Britain, 's, p...
3        [Two, Germans, and, four, Nigerian, oil, worke...
4        [The, German, firm, works, as, a, sub-contract...
                               ...                        
34274    [Indian, border, security, forces, are, accusi...
34275    [Indian, officials, said, no, one, was, injure...
34276    [Two, more, landed, in, fields, belonging, to,...
34277    [They, say, not, all, of, the, rockets, explod...
34278    [Indian, forces, said, they, responded, to, th...
Name: tokens, Length: 34279, dtype: object

## Test

In [127]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9766 - loss: 0.0831
Test Loss: 0.0740, Accuracy: 0.9781


In [128]:
# from tensorflow.keras.models import load_model

# # Load the saved model
# model = load_model('./models/ner_LSTM.keras')

In [129]:
def test_and_mask(sentence, model, word2id, label2id, id2label, MAX_LEN):
    tokens = sentence.strip().split()
    
    def encode(tokens, vocab, pad_len=MAX_LEN, fallback='<UNK>', pad_token='<PAD>'):
        fallback_id = vocab.get(fallback, list(vocab.values())[0])  
        pad_id = vocab.get(pad_token, list(vocab.values())[0])    
        ids = [vocab.get(tok, fallback_id) for tok in tokens]
        return ids[:pad_len] + [pad_id] * max(0, pad_len - len(ids))
    
    encoded_sentence = encode(tokens, word2id, pad_len=MAX_LEN)
    encoded_sentence = np.array([encoded_sentence])  
    
    # Predict using the model
    y_pred = model.predict(encoded_sentence)
    y_pred_ids = np.argmax(y_pred, axis=-1)[0] 
    
    # Decode the sentence and labels
    id2word = {v: k for k, v in word2id.items()}
    decoded_sentence = [id2word.get(word_id, '<UNK>') for word_id in encoded_sentence[0]]
    decoded_labels = [id2label[label_id] for label_id in y_pred_ids]
    
    # Mask the entities in the sentence
    masked_sentence = []
    for token, label in zip(decoded_sentence, decoded_labels):
        if token == '<PAD>':
            continue
        elif label == 'O':
            masked_sentence.append(token)
        else:
            masked_sentence.append(label)
    
    # Print the original and masked sentence
    print("Original Sentence:")
    print(" ".join(tokens))
    print("\nMasked Sentence:")
    print(" ".join(masked_sentence))

In [130]:
# Example usage
random_sentence = "Barack Obama was the president of the United States."
test_and_mask(random_sentence, model, word2id, label2id, id2label, MAX_LEN)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Original Sentence:
Barack Obama was the president of the United States.

Masked Sentence:
B-per I-per was the president of the B-geo I-org


In [131]:
# Example usage
random_sentence = "On June 15, 2023, Dr. Emily Carter, a renowned physicist from Princeton University, gave a keynote speech at the Global Climate Summit in Geneva, Switzerland. She emphasized the urgent need for international collaboration to combat climate change. Representatives from over 50 countries, including Canada, Brazil, and India, attended the event. Microsoft and Tesla announced a joint venture to develop carbon-neutral technologies. Meanwhile, UN Secretary-General António Guterres praised the summit's progress in aligning with the Paris Agreement. The summit also featured contributions from scientists at MIT and Oxford University. Elon Musk participated virtually from California, showcasing Tesla’s latest solar innovations. A follow-up summit is scheduled for November 2024 in Tokyo, Japan. Attendees shared insights via the official Twitter hashtag #GCS2023. The event concluded with a ceremonial tree planting sponsored by Green Earth Foundation."

In [132]:
batches = []
i = 1
MAX_LEN = len(random_sentence)
for i in range(0,MAX_LEN,40):
    chunk = random_sentence[i:40+i]
    string = ' '.join(chunk)
    batches.append(string)
    

In [133]:
masked_sentence_ = []
for sentence in batches:
    original_sequence,masked_sentence = test_and_mask(sentence, model,word2id, label2id,id2label,40)
    masked_sentence = ' '.join(masked_sentence)
    masked_sentence_.append(masked_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Original Sentence:
O n J u n e 1 5 , 2 0 2 3 , D r . E m i l y C a r t e r , a r e

Masked Sentence:
B-geo <UNK> <UNK> <UNK> <UNK> <UNK> B-tim I-tim , 2 <UNK> 2 3 , D <UNK> . <UNK> <UNK> I-per l <UNK> C a <UNK> <UNK> <UNK> I-per , a <UNK> <UNK>


TypeError: cannot unpack non-iterable NoneType object

In [244]:
random_sentence

['On',
 'June',
 '15,',
 '2023,',
 'Dr.',
 'Emily',
 'Carter,',
 'a',
 'renowned',
 'physicist',
 'from',
 'Princeton',
 'University,',
 'gave',
 'a',
 'keynote',
 'speech',
 'at',
 'the',
 'Global',
 'Climate',
 'Summit',
 'in',
 'Geneva,',
 'Switzerland.',
 'She',
 'emphasized',
 'the',
 'urgent',
 'need',
 'for',
 'international',
 'collaboration',
 'to',
 'combat',
 'climate',
 'change.',
 'Representatives',
 'from',
 'over',
 '50',
 'countries,',
 'including',
 'Canada,',
 'Brazil,',
 'and',
 'India,',
 'attended',
 'the',
 'event.',
 'Microsoft',
 'and',
 'Tesla',
 'announced',
 'a',
 'joint',
 'venture',
 'to',
 'develop',
 'carbon-neutral',
 'technologies.',
 'Meanwhile,',
 'UN',
 'Secretary-General',
 'António',
 'Guterres',
 'praised',
 'the',
 "summit's",
 'progress',
 'in',
 'aligning',
 'with',
 'the',
 'Paris',
 'Agreement.',
 'The',
 'summit',
 'also',
 'featured',
 'contributions',
 'from',
 'scientists',
 'at',
 'MIT',
 'and',
 'Oxford',
 'University.',
 'Elon',
 'Musk